#  Homework 2 Analysis

Run Yourself!  
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/marvins/Econ_Work/main?labpath=Analysis.ipynb)

**NOTE IF YOU ARE ON BINDER:** PLEASE HIT THE >> icon above to manually re-run.  This will fix the ipywidget table issue below.

![](./images/icon.png)

## Summary 
Forecast Colorado state marijuana tax revenue for calendar years 2024 and 2025. 
While background information is provided here, please also rely on news articles and other sources of information when preparing your forecast. 

Tax revenue data are provided along with this assignment and are summarized on a monthly and annual basis. In three pages or less, provide your forecast estimates (e.g., $\$XXX.X$ million and $\$XXX.X$ million) for calendar years 2024 and 2025 and summarize your forecast by providing an economic narrative substantiating these expectations. 

This narrative should identify key drivers of tax revenue (e.g., determinants of demand, supply, prices, and/or other factors) and your expectations for changes in these drivers.

## Background and other considerations

### 1. Marijuana taxes and tax rates. 
Three unique taxes apply differently to medical and retail marijuana. 

|              | Regular State Sales Tax |  Marijuana Retail Excise Tax | Marijuana Retail Sales Tax | 
|--------------|-------------------------|------|--------|
| Description  | Tax on most goods sold in Colorado. While medical marijuana sales are subject to the tax, retail marijuana is exempt. | Tax on the first sale or transfer of unprocessed marijuana by a retail cultivation facility. | Tax on the sale of retail marijuana or retail marijuana products to a consumer by a retail marijuana store. |
| Tax Rate | 2.9% Beginning July 1, 2017, only marijuana accessories subject to the tax, while marijuana and marijuana products are exempt. | 15% | Increased from 10% to 15% beginning July 1, 2017. |

2. Median market price data are provided with tax revenue data.

4. Legalization and taxation of marijuana in other states.
    - For a summary of states that have legalized marijuana, see: [NCSL Org](https://www.ncsl.org/research/health/state-medical-marijuana-laws.aspx).
    - For a summary of marijuana taxation across states, see: [Tax Foundation](https://taxfoundation.org/research/all/state/cannabis-tax-revenue-reform/).


## Literature

* "Marijuana Policy Overview, FY 2024-2025", Joint Budget Committee, Colorado LegislatureE  * [Link](W
https://leg.colorado.gov/sites/default/files/fy2024_-25marbrf.p)df

## Approach and Methodology

Marijuana is an interesting product which I am note sure how to categorize.  To start with, I will make the assumption that Marijuana products will likely follow the market as a whole.  

# Preparation Work

First, we load the require Python libraries and setup our logger.

In [ ]:
import pandas as pd

import plotly.graph_objects as go
import plotly.express as px

import matplotlib.pyplot as plt

import numpy as np

from ipywidgets import widgets

from sklearn.linear_model import LinearRegression

import datetime, dateutil, enum, locale, logging, os, sys

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

Logger setup

In [ ]:
logging.basicConfig( level = logging.INFO )

## Load Pre-Existing Tax Data

First, we have the monthly **regular sales tax revenue**.

In [ ]:
tax_rev_monthly_df = pd.read_excel( 'Assignment 2 - Data.xlsx',
                                    sheet_name = 'tax_revenue_monthly' )

tax_rev_monthly_df.head(3)

In [ ]:
tax_rev_yearly_df = pd.read_excel( 'Assignment 2 - Data.xlsx',
                                   sheet_name = 'tax_revenue_yearly' )

tax_rev_yearly_df.head(3)

In [ ]:
median_price = pd.read_excel( 'Assignment 2 - Data.xlsx',
                              sheet_name = 'median_market_price' )
median_price.head(3)

## Visualizing Tax Information

In [ ]:
fig = go.Figure()

fig.add_trace( go.Scatter( x = tax_rev_monthly_df['date'],
                           y = tax_rev_monthly_df['reg_state_sales_tax'],
                           name = 'Regular State Sales Taxes' ) )

fig.add_trace( go.Scatter( x = tax_rev_monthly_df['date'],
                           y = tax_rev_monthly_df['ret_mar_sales_tax'],
                           name = 'Retail Marijuana Sales Taxes' ) )

fig.add_trace( go.Scatter( x = tax_rev_monthly_df['date'],
                           y = tax_rev_monthly_df['ret_mar_exc_tax'],
                           name = 'Retail Marijuana Excise Taxes' ) )
fig.update_layout( height = 500 )

fig.show()

## Initial Observations

* **Regular State Sales Taxes** have been fairly flat since September 2017.  This matches the legislative changes which took effect in July 2017.

* **Retail Marijuana Sales Taxes** grew very quickly until early 2021.  At that point, there starts a gradual decline.  Initial speculating would point to a decrease due to COVID, tied with some larger industry trends.   Most COVID impacts should have mitigated by early 2022, meaning there are likely larger economic forces at play.

* **Retail Marijuana Excise Taxes** show a relatively common curve to the sales taxes.  This makes sense as the growers will inevitably be the suppliers for the customers.  

## Data Preparation

In order to perform regression, we need to create 2 sets of X-axis data.

1. Datetime entries (`np.datetime64`) which we plot against.  Represented as `date_range`.
2. Elapsed date entries which are just the number of days since the first entry.  This gives us an integer format we can run regression against. Represented as `date_delta`.

Note the dates extend beyond the data as this is what we run **prediction** against.  For the existing date range, simply use `data['date'] - start_date`.

In [ ]:
start_date = tax_rev_monthly_df['date'].values[0]

In [ ]:
date_range = []
date_delta = []
for year in range(2014,2026):
    for month in range( 1,13 ):
        date_range.append( np.datetime64( f'{year}-{month:02}-01' ) )
        date_delta.append( np.datetime64( f'{year}-{month:02}-01' ) - start_date)

date_range = pd.Series( date_range )
date_delta = pd.Series( date_delta ).dt.days

Now we need a function such that given a Y value (our tax revenue), we can construct a regression line estimate.

In [ ]:
class RegType(enum.Enum):
    POLYVAL = 1
    POLYFEAT = 2

In [ ]:
def build_regression( num_dims, mode, xvals_train, yvals_train, xvals_predict, start = None, end = None ):

    if start is not None and end is not None:
        xvals_train = xvals_train.iloc[ start : end ]
        yvals_train = yvals_train[ start : end ]

    # Using numpy polyfit/polyval
    if mode == RegType.POLYVAL:
        poly = np.polyfit( xvals_train.dt.days,
                  yvals_train,
                  num_dims )

        return np.polyval( poly, xvals_predict )

    #  Using Scikit-Learn
    elif mode == RegType.POLYFEAT:
        poly = PolynomialFeatures(degree=num_dims, include_bias=False)
        poly_features = poly.fit_transform( xvals_train.dt.days.values.reshape(-1, 1) )

        #fit polynomial regression model
        poly_reg_model = LinearRegression()

        poly_reg_model.fit(poly_features, yvals_train )

        #use model to make predictions on response variable
        return poly_reg_model.predict( poly.fit_transform( xvals_predict.values.astype('float64').reshape(-1,1))).reshape(xvals_predict.shape[0])


## Estimating Yearly Revenue

Given a regression line which extends through 2024 and 2025, we can estimate the full revenue for the calendar year.

In [ ]:
def estimate_year_revenue( xpred, ypred, year ):

    df = pd.DataFrame( { 'date': xpred,
                         'pred': ypred } )

    result = df.loc[df['date'] >= np.datetime64( f'{year}-01-01' )].loc[df['date'] <= np.datetime64( f'{year}-12-01' )]

    return result['pred'].sum()

## Building Plot Results

In [ ]:
dmonth = tax_rev_monthly_df['date'] - start_date

In [ ]:
dim_widget = widgets.IntSlider(
    value=1.0,
    min=1.0,
    max=12.0,
    step=1.0,
    description='Dimensions:',
    continuous_update=False
)


date_widget = widgets.SelectionRangeSlider(
    options=date_range,
    index=(2,dmonth.shape[0]-1),
    description='Months',
    disabled=False,
    layout = { 'width': '30%' },
    readout = False
)

date_html = widgets.HTML( value=( f'{np.datetime_as_string(date_range.values[date_widget.index[0]],unit="D")}, {np.datetime_as_string(date_range.values[date_widget.index[1]],unit="D")}' ) )

In [ ]:
modes = { 'reg_state_sales_tax': { 'name': 'Regular State Sales Taxes', 'color': 'red' },
          'ret_mar_sales_tax': { 'name': 'Retail Marijuana Sales Taxes', 'color': 'blue' },
          'ret_mar_exc_tax': { 'name': 'Retail Marijuana Excise Taxes', 'color': 'green' } }

maxY = 0

revenue = {}


traces = []
for mode in modes:

    kname = f'{modes[mode]["name"]} Revenue'
    revenue[kname] = { '2024': 0, '2025': 0 }
    modes[mode]['ypred'] = build_regression( int(dim_widget.value),
                                             RegType.POLYFEAT,
                                             dmonth,
                                             tax_rev_monthly_df[mode].values,
                                             date_delta,
                                             start = date_widget.index[0],
                                             end = date_widget.index[1] )

    maxY = max( np.max(tax_rev_monthly_df[mode].values), maxY )
    traces.append( go.Scatter( x = date_range,
                               y = tax_rev_monthly_df[mode].values,
                               name = modes[mode]['name'],
                               marker= dict( color = modes[mode]['color'] ),
                               mode = 'lines+markers' ))

    traces.append( go.Scatter( x = date_range,
                               y = modes[mode]['ypred'],
                               name = 'Regression Polynomial',
                               marker= dict( color = modes[mode]['color'] ),
                               mode = 'lines',
                               line = dict( dash='dot') ) )

    revenue[kname]['2024'] = estimate_year_revenue( date_range,
                                                    modes[mode]['ypred'],
                                                    2024 )
    revenue[kname]['2025'] = estimate_year_revenue( date_range,
                                                    modes[mode]['ypred'],
                                                    2025 )

revenue_df = pd.DataFrame(revenue)/1e6
revenue_df.style.format( '${:.00f} Million' )

#  Date Lines
traces.append( go.Scatter( x = pd.Series([ np.datetime64( '2024-12-01' ),np.datetime64( '2024-12-01' )]),
                           y = [-1,maxY + 10000000],
                           marker = dict( color = 'orange' ),
                           mode = 'lines',
                           name = 'Dec 2024' ) )
traces.append( go.Scatter( x = pd.Series([ np.datetime64( '2025-12-01' ),np.datetime64( '2025-12-01' )]),
                           y = [-1,maxY + 10000000],
                           marker = dict( color = 'orange' ),
                           mode = 'lines',
                           name = 'Dec 2025' ) )

#  Regression Training Lines
traces.append( go.Scatter( x = [ date_range[date_widget.index[0]], date_range[date_widget.index[0]]],
                           y = [-1,maxY + 10000000],
                           marker = dict( color = 'purple' ),
                           mode = 'lines',
                           name = 'Regression Training Bound' ) )
traces.append( go.Scatter( x = date_range,
                           y = [-1,maxY + 10000000],
                           marker = dict( color = 'purple' ),
                           mode = 'lines',
                           showlegend=False ) )

fig2 = go.FigureWidget( data = traces,
                        layout = go.Layout( title = dict( text = 'Revenue Projections' ) ) )
fig2.update_layout( height = 400 )
fig2.update_xaxes( range = [ date_range.values[0], date_range.values[-1] ] )
fig2.update_yaxes( range = [0,maxY + 1000000] )

def response( change ):

    counter = 0
    for mode in modes:
        modes[mode]['ypred'] = build_regression( int(dim_widget.value), RegType.POLYFEAT,
                                                 dmonth, tax_rev_monthly_df[mode].values,
                                                 date_delta, start = date_widget.index[0],
                                                 end = date_widget.index[1])


        rev24 = estimate_year_revenue( date_range, modes[mode]['ypred'], 2024 )/1e6
        rev25 = estimate_year_revenue( date_range, modes[mode]['ypred'], 2025 )/1e6

        kname = f'{modes[mode]["name"]} Revenue'
        revenue[kname]['2024'] = max( 0, rev24 )
        revenue[kname]['2025'] = max( 0, rev25 )
        print(revenue.keys())
        revenue_df = pd.DataFrame(revenue)/1e6
        revenue_df.style.format( '${:.00f} Million' )
        print(revenue_df)
        fig2.data[2 * counter + 1].x = date_range 
        fig2.data[2 * counter + 1].y = modes[mode]['ypred']
        counter += 1

    fig2.data[-2].x = [ date_range[date_widget.index[0]], date_range[date_widget.index[0]]]
    fig2.data[-1].x = [ date_range[date_widget.index[1]], date_range[date_widget.index[1]]]

    date_html.value = f'{np.datetime_as_string(date_range.values[date_widget.index[0]],unit="D")}, {np.datetime_as_string(date_range.values[date_widget.index[1]],unit="D")}'

dim_widget.observe( response, names="value" )
date_widget.observe( response, names="value" )

widgets.VBox([ dim_widget, widgets.HBox([date_widget,date_html]), fig2 ] )

## NOTE:  You must re-run this cell to show updated values.

In [ ]:
revenue_df.style.format('${:,.0f} Million', precision=0)

## Regular State Sales Taxes
Per the regression line **post-COVID**, <ins>Regular State Sales Taxes</ins>, 2024 will end with $\$197k$ and 2025 will end with $\$0$.  Given the sharp drop in demand, we should consider Regular State Sales Taxes for Marijuana products to approach an insignificant level.

| Regression Dimensions | Training Start Date | Training End Date  | Est Dec 31 Revenue | Est Dec 31, 2025 Revenue | Total 2024 Revenue | Total 2025 Revenue |
|-----------------------|---------------------|--------------------|--------------------|------------------|---|---|
|  1 (Linear)           |  December 31, 2021  | November 31, 2023  |   $\$197$ Million | $\$0$ Million | $\$4$ Million | $\$1$ Million |

<img src="./images/image01.png" alt="drawing" width="800"/>
<img src="./images/image02.png" alt="drawing" width="800"/>


## Retail Marijuana Sales Taxes

Retail sales taxes fare much better.  There is still a significant revenue drop, baring a return in demand.  Using a variety of options, we end up with the following revenue projections:

| Regression Dimensions | Training Start Date | Training End Date  | Est Dec 31 Revenue | Est Dec 31, 2025 Revenue | Total 2024 Revenue | Total 2025 Revenue |
|-----------------------|---------------------|--------------------|--------------------|------------------|---|---|
|  1 (Linear)           |  December 31, 2021  | November 31, 2023  |   $\$14.4$ Million | $\$12.3$ Million | $\$184$ Million | $\$159$ Million |
|  2 (Quadratic)        |  December 31, 2021  | November 31, 2023  |   $\$14.6$ Million | $\$12.8$ Million | $\$185$ Million | $\$163$ Million |

## Retail Marijuana Excise Tax

| Regression Dimensions | Training Start Date | Training End Date  | Est Dec 31 Revenue | Est Dec 31, 2025 Revenue | Total 2024 Revenue | Total 2025 Revenue |
|-----------------------|---------------------|--------------------|--------------------|------------------|---|---|
|  1 (Linear)           |  December 31, 2021  | November 31, 2023  |   $\$1.3$ Million | $\$0$ Million | $\$27$ Million | $\$3$ Million |
